In [2]:
import time
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
import AlphaNet
from AlphaNet.Data import DataLoader
from AlphaNet.Models import AlphaNet_LSTM_V1
# read_task
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()

# device = task_info["Cuda"].pop()
# task = task_info["Task"]
# task_index = task[task["status"] == "waiting"].index[0]
# Alpha_Name = task.loc[task_index,"Alpha_Name"]
# start_date = task.loc[task_index,"start_date"]
# end_date = task.loc[task_index,"end_date"]
# sequence = task.loc[task_index,"sequence"]
# LR = task.loc[task_index,"LR"]
# epoch_num = task.loc[task_index,"epoch_num"]
# feature_num = task.loc[task_index,"feature_num"]
# task.loc[task_index,"status"] = device
# np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [7]:
task_info["Factor"]

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
0,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-01-01,2019-06-01,finished,None
1,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-06-01,2020-01-01,finished,None
2,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-01-01,2020-06-01,finished,None
3,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-06-01,2021-01-01,finished,None
4,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2021-01-01,2021-06-01,finished,None
...,...,...,...,...,...,...,...,...,...
105,AlphaNetV1_Shift_5_Sequence_6_5d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,finished,"{'target': '5d_ret_standscalar', 'alpha_name':..."
106,AlphaNetV1_Shift_5_Sequence_6_5d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,finished,"{'target': '5d_ret_standscalar', 'alpha_name':..."
107,AlphaNetV1_Shift_5_Sequence_6_5d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,finished,"{'target': '5d_ret_standscalar', 'alpha_name':..."


In [8]:
target = "10d_ret_standscalar"
alpha_name = "AlphaNet_ConcatMoving5And10"
shift = 5
sequence = 6
LR = [0.001,0.00001,0.0000001]
epoch_num = [30,30,30]

In [9]:
alpha_list = [
'open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover',
#             'open_ts_norm_30d',
#              'high_ts_norm_30d',
#              'low_ts_norm_30d',
#              'close_ts_norm_30d',
#              'vwap_ts_norm_30d',
#              'volume_ts_norm_30d',
#              'pct_chg_ts_norm_30d',
#              'turnover_ts_norm_30d',
#              'free_turnover_ts_norm_30d'
             ]
feat_list = alpha_list.copy()
method = ["COV","CORR","STD","ZSCORE","RETURN","DECAY"]
from AlphaNet.Data import generate_alpha_list
alpha_list1 = generate_alpha_list(feat_list,method,day=5)
alpha_list2 = generate_alpha_list(feat_list,method,day=10)
alpha_list = []
for i in range(len(alpha_list1)):
    alpha_list.append(alpha_list1[i])
    alpha_list.append(alpha_list2[i])
alpha_list
# alpha_list = [i +"_residual" for i in feat_list]

['COV_open_high_5',
 'COV_open_high_10',
 'COV_open_low_5',
 'COV_open_low_10',
 'COV_open_close_5',
 'COV_open_close_10',
 'COV_open_vwap_5',
 'COV_open_vwap_10',
 'COV_open_volume_5',
 'COV_open_volume_10',
 'COV_open_pct_chg_5',
 'COV_open_pct_chg_10',
 'COV_open_turnover_5',
 'COV_open_turnover_10',
 'COV_open_free_turnover_5',
 'COV_open_free_turnover_10',
 'COV_high_low_5',
 'COV_high_low_10',
 'COV_high_close_5',
 'COV_high_close_10',
 'COV_high_vwap_5',
 'COV_high_vwap_10',
 'COV_high_volume_5',
 'COV_high_volume_10',
 'COV_high_pct_chg_5',
 'COV_high_pct_chg_10',
 'COV_high_turnover_5',
 'COV_high_turnover_10',
 'COV_high_free_turnover_5',
 'COV_high_free_turnover_10',
 'COV_low_close_5',
 'COV_low_close_10',
 'COV_low_vwap_5',
 'COV_low_vwap_10',
 'COV_low_volume_5',
 'COV_low_volume_10',
 'COV_low_pct_chg_5',
 'COV_low_pct_chg_10',
 'COV_low_turnover_5',
 'COV_low_turnover_10',
 'COV_low_free_turnover_5',
 'COV_low_free_turnover_10',
 'COV_close_vwap_5',
 'COV_close_vwap_10'

In [17]:
factor_df = pd.DataFrame([alpha_name,shift,sequence,LR,epoch_num,alpha_list],index=[
    'Alpha_Name','shift', 'sequence', 'LR', 'epoch_num', 'description'
]).T
factor_df.index.names = ["factor_id"]
factor_df

,Alpha_Name,shift,sequence,LR,epoch_num,description
factor_id,,,,,,
0,AlphaNet_ConcatMoving5And10,5,6,"[0.001, 1e-05, 1e-07]","[30, 30, 30]","[COV_open_high_5, COV_open_high_10, COV_open_l..."


In [23]:
task_info["CPU"] = True

In [30]:
import shutil
shutil.rmtree("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Data_lib/AlphaNetV1_TsNorm30_Shift_5_Sequence_6")

In [24]:
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)